In [11]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

In [12]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [2]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [14]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [15]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [5]:
def getValidatorAndDelegationRewards():
    acc_rewards = dict()
    delegations = dict()


In [16]:
def R4_test(single):
    logger.info(f"Test-R4: Reward given out to delegators sums up to the total delegation reward for each validator")
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block + 1 == last_block:
        logger.info(f"current at the last block or last second block, wait until the 5th/6th block in the new epoch")
        while block < last_block+6:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")
    acc_rewards_prev = dict()
    delegations_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            ds = i['validator']['delegations']
            dels = dict()
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                dels[d_addr] = d_reward
            delegations_prev[address] = dels  
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block reached, {block}, will begin testing...")
    flag = True
    logger.info(f"current block: {block}")
    # get the validator info and compute validator rewards
    acc_rewards_curr = dict()
    delegations_curr = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_curr[address] = reward_accumulated
            if address not in acc_rewards_prev:
                continue
            reward = reward_accumulated - acc_rewards_prev[address]
            if reward == 0:
                continue
            del_rewards = 0
            dels = delegations_prev[address]
            ds = i['validator']['delegations']
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                del_rewards += d['reward']
                if d_addr in dels:
                    del_rewards -= dels[d_addr]
            if format(del_rewards, '.20e') != format(reward, '.20e'):
                logger.warning(f"Test-R4:Fail")
                logger.warning(f"for validator {address}, validator reward: {reward:.20e}, delegators reward: {del_rewards:.20e}\n")
                flag = False
    if single:
        curr_test = None
    else:       
        curr_test = R5_test
    if flag:
        logger.info(f"Test-R4: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test
    

In [18]:
R4_test(True)

INFO:economic-test:Test-R4: Reward given out to delegators sums up to the total delegation reward for each validator
INFO:economic-test:current and last block numbers: 3586553, 3588095
INFO:economic-test:current block 3586553, will begin collecting infos...


current and last block numbers 3586553 3588095


INFO:economic-test:new block reached, 3586554, will begin testing...
INFO:economic-test:current block: 3586554
INFO:economic-test:Test-R4: Succeed



(True, None)